# Variability Heatmap (Detail)

VariabilityHeatmapDetail ではほとんど協力が進化するので、変動性による影響が出なかったのではないか？という仮説のもと、少し協力が起こりにくい利得構造で試してみる。

その結果、この実験 (VariabilityHeatmapDetail2) でもほとんど協力が進化し、変動性による影響はあまり大きく出なかった。しかし、変動性が高まればグループサイズが大きくなるという傾向は、わずかながらはっきりと見られる。特にstrong_component1で顕著に見られる。

次は、個体数が少なすぎて協力が進化しやすいのではないかという仮説のもと、個体数を300に増やして利得構造は変えずにもう一度やってみる。利得構造は変えないので、同じノートブックで検証する。変動性が高まればグループサイズが大きくなるという傾向は、わずかながらはっきりと見られる。

もっとはっきりと見たいので、個体数300で利得構造を(1.2, 1.3) (-0.2, -0.3) で試してみる。→ VariabilityHeatmapDetail3

# 結論 (仮)

- 非常に協力が進化しやすい利得構造において、変動性は強い繋がりによるグループ形成を促進する。
- ある程度協力が進化しやすい利得構造において、変動性は、協力率、強い繋がりによるグループ形成、中程度の繋がりによるグループ形成を促進する。
- あまり協力が進化しやすくない利得構造においては、変動性はグループ形成に影響を与えない。

# 議論

- なぜ、ある程度協力が進化しやすい利得構造において、変動性は、協力率、強い繋がりによるグループ形成、中程度の繋がりによるグループ形成を促進するのか？
    - 人口減少局面では、適応度が低い個体がどんどん死んでいく。
    - 適応度が高い個体は子孫を残せる。強い繋がりの個体とのグループを形成できる。
    - 異常な人口減少局面では、協力が進化しやすい。協力が進化すると、繋がりが強くなる。その後、人口が増えてくると、その繋がりが維持されたまま、新しいヒトが入ってきても、旧レジームの人たちは安泰なまま。
    - つまり、大きく人口が減る局面が、協力的なグループを作るチャンス。
        - なぜ、人口が少ないと協力が進化しやすいのか？
- なぜ、非常に協力が進化しやすい利得構造において、変動性は強い繋がりによるグループ形成を促進するのか？

In [ ]:
using DataFrames: DataFrame, AbstractDataFrame, groupby
using Plots: Plot, plot, heatmap
using GLM

include("../src/SimPlot.jl")
using .SimPlot

In [ ]:
function plot_heatmap(df::AbstractDataFrame, symbol::Symbol)::Plot
    β = sort(unique(df.β))
    σ = sort(unique(df.σ))

    return heatmap(
        σ,
        β,
        reshape([SimPlot.get_value(df, s, b, :σ, :β, symbol) for b in β, s in σ], length(β), length(σ)),
        xlabel = "σ",
        ylabel = "β",
        xlims = (-0.025, 1.025),
        ylims = (-0.025, 1.025),
        xticks = 0:0.1:1,
        yticks = 0:0.1:1,
        title = symbol,
    )
end

function make_asterisk(value::Float64)::String
    if value < 0.001
        "***"
    elseif value < 0.01
        "**"
    elseif value < 0.05
        "*"
    else
        ""
    end
end

function fit_and_extract(formula::FormulaTerm, df::AbstractDataFrame)::DataFrame
    lm_fit = lm(formula, df)
    _coef = coef(lm_fit)
    _pvals = coeftable(lm_fit).cols[4]  # Assuming p-values are in column 4

    return DataFrame(
        formula = string(formula),
        Intercept = _coef[1],
        β = _coef[2],
        σ = _coef[3],
        p_β = _pvals[2],
        p_σ = _pvals[3],
        eval_β = make_asterisk(_pvals[2]),
        eval_σ = make_asterisk(_pvals[3]),
        impact_β = _coef[2] / _coef[1] > 0.1 ? round(_coef[2] / _coef[1], digits = 2) : "",
        impact_σ = _coef[3] / _coef[1] > 0.1 ? round(_coef[3] / _coef[1], digits = 2) : "",
    )
end

formula_vec = [
    @formula(cooperation_rate ~ β + σ),
    @formula(weak_component1_count ~ β + σ),
    @formula(weak_component1_size_μ ~ β + σ),
    @formula(weak_component1_size_max ~ β + σ),
    @formula(weak_component2_count ~ β + σ),
    @formula(weak_component2_size_μ ~ β + σ),
    @formula(weak_component2_size_max ~ β + σ),
    @formula(medium_component1_count ~ β + σ),
    @formula(medium_component1_size_μ ~ β + σ),
    @formula(medium_component1_size_max ~ β + σ),
    @formula(medium_component2_count ~ β + σ),
    @formula(medium_component2_size_μ ~ β + σ),
    @formula(medium_component2_size_max ~ β + σ),
    @formula(strong_component1_count ~ β + σ),
    @formula(strong_component1_size_μ ~ β + σ),
    @formula(strong_component1_size_max ~ β + σ),
    @formula(strong_component2_count ~ β + σ),
    @formula(strong_component2_size_μ ~ β + σ),
    @formula(strong_component2_size_max ~ β + σ),
];

In [ ]:
# 1 100個体: "20230927_130307", "20230927_133154", "20230927_134634",
# 1 200個体: "20230927_143216", "20230927_154806",
# 2 200個体 (PD 協力が進化しにくい利得構造で実験)
# "20230927_191311", "20230927_201430", "20230927_211529", "20230927_221850", "20230927_232049",
# 2 300個体 (PD 協力が進化しにくい利得構造で実験)
# "20230928_061443", "20230928_091423", "20230928_114521", "20230928_122355", "20230928_152820",
# 3 300個体 (PD 協力が進化しにくい利得構造で実験)
# "20230928_201655", "20230928_231027", "20230929_020115", "20230929_045809", "20230929_080339",
df_vec = SimPlot.csv_to_df(["VariabilityHeatmapDetail"]);

In [ ]:
mean_df = SimPlot.make_mean_df(df_vec)
@show size(mean_df)
df_vec = nothing;

In [ ]:
for df in groupby(sort(mean_df, [:T, :S, :initial_N, :generations]), [:initial_N, :generations, :T, :S])
    suptitle = "T: $(df.T[1]), S: $(df.S[1]), Intial population: $(df.initial_N[1]), generations: $(df.generations[1])"
    
    cooperation_heatmap = plot_heatmap(df, :cooperation_rate)
    empty_plot = plot(legend=false, grid=false, axis=false)
    heatmap_vec = [plot_heatmap(df, Symbol(col)) for col in names(df)[15:end]]

    display(plot(cooperation_heatmap, empty_plot, empty_plot, heatmap_vec..., 
            layout = (13, 3), size = (1500, 2400), suptitle = suptitle))

    display(vcat([fit_and_extract(formula, df) for formula in formula_vec]...))
end;

In [ ]:
for df in groupby(mean_df, [:T, :S])
    suptitle = "T: $(df.T[1]), S: $(df.S[1])"
    
    cooperation_heatmap = plot_heatmap(df, :cooperation_rate)
    empty_plot = plot(legend=false, grid=false, axis=false)
    heatmap_vec = [plot_heatmap(df, Symbol(col)) for col in names(df)[15:end]]

    display(plot(cooperation_heatmap, empty_plot, empty_plot, heatmap_vec..., 
            layout = (13, 3), size = (1500, 2400), suptitle = suptitle))

    display(vcat([fit_and_extract(formula, df) for formula in formula_vec]...))
end;

In [ ]:
using StatsBase: mean
using DataFrames: combine, groupby
using Plots: plot!

agg_df = combine(groupby(mean_df_PD, [:β, :σ]), :cooperation_rate => mean => :mean_cooperation_rate)

p = plot(xticks = 0:0.1:1, xl = "σ", yl = "cooperation rate", title = "Plot by β values")

for subdf in groupby(agg_df, :β)
    plot!(subdf.σ, subdf.mean_cooperation_rate, label=false)
end

overall_mean_df = combine(groupby(agg_df, :σ), :mean_cooperation_rate => mean => :mean_cooperation_rate)
plot!(overall_mean_df.σ, overall_mean_df.mean_cooperation_rate, line = (:black, 3), label=false)

display(p)